In [360]:
# coding: utf-8
"""
igraphを使ったグラフクラスタリング
greedyアルゴリズム（louvain法）によってクラスタリングした後
サブグラフの中心性指標を図ることで単語の順位付けを行う
媒介中心性と固有ベクトル中心性の２種類を採用
"""
from igraph import *
import csv
import collections

# csvファイルの読み込み
def readcsv(path):
    f = open(path, "rb")
    dataReader = csv.reader(f)
    arr = [row for row in dataReader]
    return arr

def writecsv(arr, path):
    f = open(path, "ab")
    dataWriter = csv.writer(f)
    dataWriter.writerows(arr)
    f.close()

# 有向エッジリストを入力して、重み付き無向ネットワークを出力する
def cal_edgelist_to_network(list_edge):
    # 有向エッジリストを無向エッジリストに変換する
    list_edge = [tuple(sorted(row)) for row in list_edge]
    # ノードリスト
    list_vertices = list(set([word for row in list_edge for word in row]))
    # エッジリストとそのweightを作成
    tuple_edge, tuple_weight = zip(*collections.Counter(list_edge).items())
    return {"vertex": list_vertices, "edge": list(tuple_edge), "weight": list(tuple_weight)}

# クラスタリング済みのネットワークを元にサブグラフのリストを作成
def cal_cluster_to_network(dict_network):
    if dict_network.has_key("cluster") == False:
        print "クラスタリングができていません"
    
    # クラスタごとにwordをまとめる
    dict_cluster = collections.defaultdict(list)
    for word, cluster in zip(dict_network["vertex"], dict_network["cluster"]):
        dict_cluster[cluster].append(word)

    # リストに変換
    list_cluster_vertex = [row[1] for row in dict_cluster.items()]
    
    # 同様にエッジとウェイトのリストも作成する
    list_cluster_edge = []
    list_cluster_weight = []
    for cluster_vertex in list_cluster_vertex:
        list_cluster_edge_one = []
        list_cluster_weight_one = []
        for row, weight in zip(dict_network["edge"], dict_network["weight"]):
            if row[0] in cluster_vertex and row[1] in cluster_vertex:
                list_cluster_edge_one.append(row)
                list_cluster_weight_one.append(weight)
        list_cluster_edge.append(list_cluster_edge_one)
        list_cluster_weight.append(list_cluster_weight_one)
    
    # まとめる
    list_dict_network = [{"vertex": cluster_vertex,
                          "edge": cluster_edge,
                          "weight": cluster_weight}
                         for cluster_vertex, cluster_edge, cluster_weight
                         in zip(list_cluster_vertex, list_cluster_edge, list_cluster_weight)]
    
    return list_dict_network

メイン部分

In [363]:
# エッジリストの読み込み
list_edge = readcsv("./files/text_for_cytoscape_rev.csv")
# 元のネットワークを作成する（無向）
dict_network_master = cal_master_network(list_edge)
# g = Graph(directed=True)
g_master = Graph()
g_master.add_vertices(dict_network_master["vertex"])
g_master.add_edges(dict_network_master["edge"])
# louvain法によるクラスタリング、vertexと同じ長さのクラスタ番号が書かれたリストがreturn
dict_network_master["cluster"] = g_master.community_fastgreedy(weights=dict_network_master["weight"]).as_clustering().membership
# クラスタ結果をもとにサブグラフのリストを作成
list_dict_network_sub = cal_cluster_to_network(dict_network_master)
# サブクラスタごとに中心性の計算
for i, dict_network_sub in enumerate(list_dict_network_sub):
    g_sub = Graph()
    g_sub.add_vertices(dict_network_sub["vertex"])
    g_sub.add_edges(dict_network_sub["edge"])
    list_dict_network_sub[i]["center_bet"] = g_sub.betweenness(directed=False, weights=dict_network_sub["weight"])
    list_dict_network_sub[i]["center_eigen"] = g_sub.eigenvector_centrality(directed=False, weights=dict_network_sub["weight"])

プロット部分

In [47]:
# クラスタ番号 
num = 0

list_word_bet = sorted(zip(list_dict_network_sub[num]["vertex"], list_dict_network_sub[num]["center_bet"]), key=lambda x: x[1], reverse=True)
list_word_eigen = sorted(zip(list_dict_network_sub[num]["vertex"], list_dict_network_sub[num]["center_eigen"]), key=lambda x: x[1], reverse=True)

for word in zip(list_word_bet, list_word_eigen):
    print word[0][0], "|||||", word[1][0]

サービス ||||| サービス
高い ||||| 高い
楽しい ||||| 充実
行事 ||||| 設備
参加 ||||| 内容
生活 ||||| 行事
充実 ||||| 金額
季節 ||||| 価格
知人 ||||| 生活
コンサート ||||| 安い
設備 ||||| 非常
本人 ||||| 妥当
満足 ||||| 季節
可能 ||||| 参加
価格 ||||| 楽しい
金額 ||||| 納得
地域 ||||| 満足
楽しみ ||||| 毎日
他 ||||| 値段
要介護 ||||| 質
外出 ||||| 知人
非常 ||||| 本人
内容 ||||| 設定
月 ||||| 他
月額 ||||| 様々
様子 ||||| 様子
趣味 ||||| 高額
料金 ||||| サークル
お金 ||||| 素晴らしい
嬉しい ||||| 料金
程度 ||||| 可能
様々 ||||| 高め
クリスマス ||||| 開催
負担 ||||| 相応
お花見 ||||| 食費
毎日 ||||| レベル
歌 ||||| 活動
納得 ||||| 天井
値段 ||||| 楽しみ
方途 ||||| コンサート
教室 ||||| 総合
話 ||||| 考慮
父 ||||| 月額
入所 ||||| 要介護
退屈 ||||| 入浴
高額 ||||| それなり
少ない ||||| 月
素晴らしい ||||| 自立
体 ||||| 旅行
手芸 ||||| 趣味
安い ||||| 地域
企画 ||||| 価値
意味 ||||| 話
月々 ||||| 居宅
映画 ||||| 管理費
我が家 ||||| 企画
家賃 ||||| 嬉しい
ボランティア ||||| 月々
サークル ||||| 外出
バスツアー ||||| 各種
認知症 ||||| 満足度
不自由 ||||| 評価
開催 ||||| 平均
お話 ||||| 教室
花見 ||||| 介護施設
妥当 ||||| お話
活動 ||||| 住環境
洗濯 ||||| 季節感
季節感 ||||| 体操
年金 ||||| お花見
夏祭り ||||| 意味
催し ||||| 父
それなり ||||| 水準
一緒 ||||| クリーニング
交流 ||||| 入所
鑑賞 ||||| 認知症
質 ||||| 退屈
アクティビティ ||||| アクティビティ
同士 |